In [ ]:
import pandas as pd

In [ ]:
df = pd.read_excel('files/inconsistencias-shipped_at.xlsx')
df

In [ ]:
df = df.sort_values(by="bundle_id", ascending=False)

In [ ]:
import http.client
import json

from dotenv import load_dotenv
import os

load_dotenv()

# only correios
import xmltodict


In [ ]:
def jadlog(code: str) -> bool:
    conn = http.client.HTTPSConnection("www.jadlog.com.br")

    headersList = {
        "Authorization": "Bearer %s" % os.getenv('JADLOG_TOKEN'),
        "Content-Type": "application/json" 
    }

    payload = json.dumps({"consulta": [{"shipmentId": code}]})

    conn.request("GET", "/api/tracking/consultar", payload, headersList)
    response = conn.getresponse()
    result = response.read()

    for obj in json.loads(result.decode('utf-8'))['consulta']:
        if len(obj['tracking']['eventos']) > 1:
            return True
    return False
 
def kangu(code: str) -> bool:
    conn = http.client.HTTPSConnection("portal.kangu.com.br")

    headersList = {
        "Accept": "*/*",
        "token": os.getenv('KANGU_TOKEN')
    }

    url = f'/tms/transporte/rastrear/{code.replace(" ", "")}'
    conn.request("GET", url, "", headersList)

    response = conn.getresponse()
    if response.status != 200:
        return False

    result = response.read()
    return True if len(json.loads(result.decode('utf-8'))['historico']) > 1 else False

def correios(code: str) -> bool:

    conn = http.client.HTTPSConnection("webservice.correios.com.br")

    headersList = {
        "Content-Type": "text/xml; charset=utf-8",
        "SOAPAction": "buscaEventos" 
    }

    usr = os.getenv('CORREIOS_USR')
    pwd = os.getenv('CORREIOS_PWD')
    payload = f'<?xml version="1.0" encoding="ISO-8859-1"?>\n<SOAP-ENV:Envelope\n xmlns:SOAP-ENV="http://schemas.xmlsoap.org/soap/envelope/"\n xmlns:ns="http://resource.webservice.correios.com.br/"\n xmlns:tns="http://resource.webservice.correios.com.br/"\n xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">\n <SOAP-ENV:Body>\n <tns:buscaEventos>\n <usuario>{usr}</usuario>\n <senha>{pwd}</senha>\n <tipo>L</tipo>\n <resultado>T</resultado>\n <lingua>101</lingua>\n <objetos>{code}</objetos>\n </tns:buscaEventos>\n </SOAP-ENV:Body>\n</SOAP-ENV:Envelope>'

    conn.request("POST", "/service/rastro", payload, headersList)
    response = conn.getresponse()
    result = response.read()

    result = xmltodict.parse(result.decode("utf-8"))

    key = result['soapenv:Envelope']['soapenv:Body']['ns2:buscaEventosResponse']['return']['objeto']
    return True if 'evento' in key and len(key['evento']) > 0 else False

In [ ]:
consultas = []

for index, row in df.iterrows():
    response = eval('%s("%s")' % (row['shipping_service'], row['tracking_code']))

    consultas.append(response)
    print(f"tracking_code: {row['tracking_code']}, servico: {row['shipping_service']}, r: {response}")

df = df.assign(has_tracking=consultas)

In [ ]:
df.to_csv('files/inconsistencias-shipped_at.csv', index=False)


In [ ]:
bundles = ', '.join(map(str, df['bundle_id'].to_list()))

query = f"""SELECT 
    unit_logs.action,
    unit_logs.created_at,
    unit_logs.unit_id,
    unit_logs.user_id,
    units.state unit_state
FROM unit_logs 
INNER JOIN units ON units.id = unit_logs.unit_id
WHERE units.bundle_id IN ({bundles})
"""

print(query)
